###HW2.0:
How do you merge  two sorted  lists/arrays of records of the form [key, value]? Where is this  used in Hadoop MapReduce? [Hint within the shuffle]
What is  a combiner function in the context of Hadoop? 
Give an example where it can be used and justify why it should be used in the context of this problem.
What is the Hadoop shuffle?

If this lists are pre-sorted, you can loop through each list and identify which value at the 0 position is smallest.  Then once you have identified that, you return the value, pop off the position and move the pointer.  This process is now repeated with the pointer of the list that returned a value at the next placement.  This process is repeated until all positions have been returned.  This is used when hadoop sends data from the mappers to the reducers.

A combiner is used to combine all of the information from the mappers before they are released to the reducers.  This is used as a barrier before the reducers can run to ensure that all the information has been collected before reducer tasks are started.

If different information is located on more than 1 mapper, we need to make sure that all of the information is sorted together before it's sent to the reducer.  For example, say that all the last names from Minnesota are in mapper 1, and all last names from Wisconsin are in Mapper 2, we need to sort both lists in the combiner to get the global sort for all records in both mappers.

Hadoop shuffle is the process of gathering data from mappers, sorting it, and sending it to the reducer.

###HW2.1: Counters as a debugging aid (and for getting work done, but please use sparingly as they are heavy)

Consumer complaints dataset: Use Counters to do EDA (exploratory data analysis and to monitor progress)
Counters are lightweight objects in Hadoop that allow you to keep track of system progress in both the map and reduce stages of processing. By default, Hadoop defines a number of standard counters in "groups"; these show up in the jobtracker webapp, giving you information such as "Map input records", "Map output records", etc. 

While processing information/data using MapReduce job, it is a challenge to monitor the progress of parallel threads running across nodes of distributed clusters. Moreover, it is also complicated to distinguish between the data that has been processed and the data which is yet to be processed. The MapReduce Framework offers a provision of user-defined Counters, which can be effectively utilized to monitor the progress of data across nodes of distributed clusters.

Use the Consumer Complaints  Dataset provide here to complete this question:

     https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0

Use the following command to grab the file:

    curl -L https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0 -o Consumer_Complaints.csv

The consumer complaints dataset consists of diverse consumer complaints, which have been reported across the United States regarding various types of loans. The dataset consists of records of the form:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

Here’s is the first few lines of the  of the Consumer Complaints  Dataset:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,
1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,
1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,

User-defined Counters

Now, let’s use MapReduce Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).

Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. 


In [1]:
!curl -L https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0 > Consumer_Complaints.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 48.5M  100 48.5M    0     0  1324k      0  0:00:37  0:00:37 --:--:-- 1560k


In [2]:
!head Consumer_Complaints.csv

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,
1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,
1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1115890,Credit reporting,,Incorrect information on credit report,Information is

In [7]:
%%writefile mr_wc_counter.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
 

WORD_RE = re.compile(r"[\w']+")
 
class MRWordFreqCount(MRJob):
    def mapper(self, _, line):
        line = line.strip().lower().split(",")
        category = line[1]
        if category != 'product':
            if category in ['debt collection', 'mortgage']:
                self.increment_counter('Product Type', category, 1)
            else:
                self.increment_counter('Product Type', 'other', 1)

    #def reducer(self, category, counts):
    #    self.increment_counter('group', 'Num_reducer_calls', 1)
    #    if category == 'debt collection':
    #        yield category, sum(counts)
    #    elif category == 'mortgage':
    #        yield category, sum(counts)
    #    else:
    #        yield category

if __name__ == '__main__':
    MRWordFreqCount.run()

Overwriting mr_wc_counter.py


In [8]:
!python mr_wc_counter.py Consumer_Complaints.csv

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/mr_wc_counter.z013nx1.20160706.174758.406218
Running step 1 of 1...
Counters: 3
	Product Type
		debt collection=44372
		mortgage=125752
		other=142788
Streaming final output from /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/mr_wc_counter.z013nx1.20160706.174758.406218/output...
Removing temp directory /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/mr_wc_counter.z013nx1.20160706.174758.406218...


In [9]:
from mr_wc_counter import MRWordFreqCount
mr_job = MRWordFreqCount(args=['Consumer_Complaints.csv'])
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access of the output 
    print runner.counters()

[{'Product Type': {'debt collection': 44372, 'other': 142788, 'mortgage': 125752}}]


# HW2.2: Analyze the performance of your Mappers, Combiners and Reducers using Counters

#### Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.

In [15]:
!echo foo foo quux labs foo bar quux > WordCount.txt

In [16]:
%%writefile mr_wc_counter.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
 
WORD_RE = re.compile(r"[\w']+")
 
class MRWordFreqCount(MRJob):
    def mapper(self, _, line):
        self.increment_counter('group', 'Num_mapper_calls', 1)
        for word in WORD_RE.findall(line):
            yield word.lower(), 1

    def reducer(self, word, counts):
        self.increment_counter('group', 'Num_reducer_calls', 1)
        yield word, sum(counts)

if __name__ == '__main__':
    MRWordFreqCount.run()

Overwriting mr_wc_counter.py


In [18]:
!python mr_wc_counter.py WordCount.txt

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/mr_wc_counter.z013nx1.20160706.175715.251281
Running step 1 of 1...
Counters: 1
	group
		Num_mapper_calls=1
Counters: 2
	group
		Num_mapper_calls=1
		Num_reducer_calls=4
Streaming final output from /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/mr_wc_counter.z013nx1.20160706.175715.251281/output...
"bar"	1
"foo"	3
"labs"	1
"quux"	2
Removing temp directory /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/mr_wc_counter.z013nx1.20160706.175715.251281...


Mapper Counter = 1 and Reducer Count = 4.  1 map task to get all of the global counts and 4 reduce tasks for the 4 distinct words.

#### Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many times the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

In [51]:
%%writefile mr_wc_counter2.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
 

WORD_RE = re.compile(r"[\w']+")
 
class MRWordFreqCount(MRJob):
    def mapper(self, _, line):
        self.increment_counter('group', 'Num_mapper_calls', 1)
        line = line.strip().lower().split(",")
        category = line[3]
        words = category.split(" ")
        if words != 'issue':
            for word in words:
                yield word, 1

    def reducer(self, word, counts):
        self.increment_counter('group', 'Num_reducer_calls', 1)
        yield word, sum(counts)

if __name__ == '__main__':
    MRWordFreqCount.run()

Overwriting mr_wc_counter2.py


In [52]:
!python mr_wc_counter2.py Consumer_Complaints.csv

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/mr_wc_counter2.z013nx1.20160706.183125.566862
Running step 1 of 1...
Counters: 1
	group
		Num_mapper_calls=312913
Counters: 2
	group
		Num_mapper_calls=312913
		Num_reducer_calls=172
Streaming final output from /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/mr_wc_counter2.z013nx1.20160706.183125.566862/output...
""	12549
"/"	12386
"\"account"	16205
"\"application"	8625
"\"loan"	107254
"\"making/receiving"	3226
"a"	3503
"account"	4476
"acct"	163
"action"	2505
"advance"	240
"advertising"	1193
"amount"	98
"amt"	71
"an"	2505
"and"	16448
"application"	243
"applied"	139
"apply"	118
"apr"	3431
"arbitration"	168
"are"	3821
"atm"	2422
"attempts"	11848
"available"	274
"balance"	597
"bank"	202
"bankruptcy"	222
"being"	5663
"billing"	8158
"by"	5663
"can't"	1999
"card"	4405
"cash"	240
"caused"	5663
"changes"	350
"charged"	976
"charges"	131
"checks"	75
"closing/cancelling"	

Number of mapper calls = 312, 913 and number of reducer calls = 172

#### Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, combiner counter, and Reducer Counter after completing your word count job.

In [54]:
%%writefile mr_wc_counter3.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
 

WORD_RE = re.compile(r"[\w']+")
 
class MRWordFreqCount(MRJob):
    def mapper(self, _, line):
        self.increment_counter('group', 'Num_mapper_calls', 1)
        line = line.strip().lower().split(",")
        category = line[3]
        words = category.split(" ")
        if words != 'issue':
            for word in words:
                yield word, 1
    def combiner(self, word, count):
        self.increment_counter('group', 'Num_combiner_calls', 1)
        yield word, sum(count)

    def reducer(self, word, counts):
        self.increment_counter('group', 'Num_reducer_calls', 1)
        yield word, sum(counts)

if __name__ == '__main__':
    MRWordFreqCount.run()

Writing mr_wc_counter3.py


In [55]:
!python mr_wc_counter3.py Consumer_Complaints.csv

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/mr_wc_counter3.z013nx1.20160706.183503.838062
Running step 1 of 1...
Counters: 2
	group
		Num_combiner_calls=319
		Num_mapper_calls=312913
Counters: 3
	group
		Num_combiner_calls=319
		Num_mapper_calls=312913
		Num_reducer_calls=172
Streaming final output from /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/mr_wc_counter3.z013nx1.20160706.183503.838062/output...
""	12549
"/"	12386
"\"account"	16205
"\"application"	8625
"\"loan"	107254
"\"making/receiving"	3226
"a"	3503
"account"	4476
"acct"	163
"action"	2505
"advance"	240
"advertising"	1193
"amount"	98
"amt"	71
"an"	2505
"and"	16448
"application"	243
"applied"	139
"apply"	118
"apr"	3431
"arbitration"	168
"are"	3821
"atm"	2422
"attempts"	11848
"available"	274
"balance"	597
"bank"	202
"bankruptcy"	222
"being"	5663
"billing"	8158
"by"	5663
"can't"	1999
"card"	4405
"cash"	240
"caused"	5663
"changes"	350
"charged"	9

Number of mapper calls = 312, 913, number of combiner calls = 319 and number of reducer calls = 172

### HW2.2.1: 
### Using a single reducer perform a sort of the words in decreasing order of word counts. Present the top 50 terms and their frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

HINT: You will need a second MRStep for the sort part. Step 1 will be the usual word count, while step 2 will be a sort step. Please use the Hadoop/MRJob framework to perform the sort. Please do NOT use any of the built-in sorts  from  python.

In [135]:
%%writefile mr_wc_counter4.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
 

WORD_RE = re.compile(r"[\w']+")
 
class MRWordFreqCount(MRJob):
    def mapper(self, _, line):
        self.increment_counter('group', 'Num_mapper_calls', 1)
        line = line.strip().lower().split(",")
        category = line[3]
        words = category.split(" ")
        if words != 'issue':
            for word in words:
                yield word, 1
    
    def reducer(self, word, counts):
        self.increment_counter('group', 'Num_reducer_calls', 1)
        yield word, sum(counts)

    def newmapper(self, key, value):
        self.increment_counter('group', 'Num_mapper_calls', 1)
        yield None, (value, key)
    
    def newreducer(self, key, values):
        self.increment_counter('group', 'Num_reducer_calls', 1)
        topfifty = []
        bottomten = []
        for count, word in values:
            topfifty.append((count, word))
            bottomten.append((count, word))
            topfifty.sort()
            bottomten.sort(reverse=True)
            topfifty = topfifty[-50:]
            bottomten = bottomten[-10:]
            topfifty.sort(reverse=True)
            bottomten.sort()
            
        for count, word in topfifty:
            yield None, (count, word)
        for count, word in bottomten:
            yield None, (count, word)

    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                  combiner=self.reducer,
                  reducer=self.reducer),
            MRStep(mapper=self.newmapper,
                  combiner=self.newreducer,
                  reducer=self.newreducer) ]
    
if __name__ == '__main__':
    MRWordFreqCount.run()

Overwriting mr_wc_counter4.py


In [136]:
!python mr_wc_counter4.py Consumer_Complaints.csv

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/mr_wc_counter4.z013nx1.20160706.200734.941471
Running step 1 of 2...
Counters: 2
	group
		Num_mapper_calls=312913
		Num_reducer_calls=319
Counters: 2
	group
		Num_mapper_calls=312913
		Num_reducer_calls=491
Running step 2 of 2...
Counters: 2
	group
		Num_mapper_calls=172
		Num_reducer_calls=2
Counters: 2
	group
		Num_mapper_calls=172
		Num_reducer_calls=3
Streaming final output from /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/mr_wc_counter4.z013nx1.20160706.200734.941471/output...
null	[107254, "\"loan"]
null	[70487, "modification"]
null	[50894, "credit"]
null	[36767, "servicing"]
null	[30546, "report"]
null	[29069, "on"]
null	[29069, "information"]
null	[29069, "incorrect"]
null	[22533, "or"]
null	[19309, "debt"]
null	[16448, "and"]
null	[16205, "opening"]
null	[16205, "\"account"]
null	[12549, ""]
null	[12545, "health"]
null	[12545, "club"]
null	[12386, "

### HW2.2.2:   
### Repeat HW2.2.1 using 3 reducers. Use the same code as in HW2.2.1  with just one modification 
to the command line: just add --jobconf mapred.reduce.tasks=3 as see presented here: 

    python HW2.2WordCount.py --jobconf mapred.reduce.tasks=3 oneLinerTextFile.txt

Describe what you see. Is this correct?

HW2.2.3: [Optional; we will cover this in class]  
Solve the "total sort" issue in HW2.2.2. 

In [137]:
!python mr_wc_counter4.py --jobconf mapred.reduce.tasks=3 Consumer_Complaints.csv

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/mr_wc_counter4.z013nx1.20160706.200806.544821
Running step 1 of 2...
Counters: 2
	group
		Num_mapper_calls=312913
		Num_reducer_calls=319
Counters: 2
	group
		Num_mapper_calls=312913
		Num_reducer_calls=491
Running step 2 of 2...
Counters: 2
	group
		Num_mapper_calls=172
		Num_reducer_calls=3
Counters: 2
	group
		Num_mapper_calls=172
		Num_reducer_calls=4
Streaming final output from /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/mr_wc_counter4.z013nx1.20160706.200806.544821/output...
null	[107254, "\"loan"]
null	[70487, "modification"]
null	[50894, "credit"]
null	[36767, "servicing"]
null	[30546, "report"]
null	[29069, "on"]
null	[29069, "information"]
null	[29069, "incorrect"]
null	[22533, "or"]
null	[19309, "debt"]
null	[16448, "and"]
null	[16205, "opening"]
null	[16205, "\"account"]
null	[12549, ""]
null	[12545, "health"]
null	[12545, "club"]
null	[12386, "

When forcing the reducers to 3, it does 1 more reducer job in the first reducer then when it was left to one.  The total reducer count is 4 instead of 3 because of this

# HW2.3: Shopping Cart Analysis

Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.
  
For this homework use the online browsing behavior dataset located at: 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.

Here are the first few lines of the ProductPurchaseData 
FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 


Do some exploratory data analysis of this dataset guided by the following questions:. 

How many unique items are available from this supplier?

Using a single reducer: Report your findings: such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,   (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 

2.3.1 OPTIONAL 
Using 2 reducers:  Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 

In [138]:
!curl -L https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0 > browse_data.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
100 3377k  100 3377k    0     0   526k      0  0:00:06  0:00:06 --:--:-- 1265k


In [139]:
!head browse_data.txt

FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 
ELE17451 GRO73461 DAI22896 SNA99873 FRO18919 DAI50921 SNA80192 GRO75578 
ELE17451 ELE59935 FRO18919 ELE23393 SNA80192 SNA85662 SNA91554 DAI22177 
ELE17451 SNA69641 FRO18919 SNA90258 ELE28573 ELE11375 DAI14125 FRO78087 
ELE17451 GRO73461 DAI22896 SNA80192 SNA85662 SNA90258 DAI46755 FRO81176 ELE66810 DAI49199 DAI91535 GRO94758 ELE94711 DAI22177 
ELE17451 SNA69641 DAI91535 GRO94758 GRO99222 FRO76833 FRO81176 SNA80192 DAI54690 ELE37798 GRO56989 


In [198]:
%%writefile basket.py
from mrjob.job import MRJob
from mrjob.step import MRStep
 
class MRWordFreqCount(MRJob):
        
    def mapper(self, word, line):
        line = line.strip().split(" ")
        for word in line:
            yield word.lower(), 1

    def combiner(self, word, counts):
        yield word, sum(counts)

    def reducer(self, word, counts):
        yield word, sum(counts)
    
    def mapper_new(self, word, counts):
        yield None, (counts, word)
 
    def reducer_new(self, word, counts):
        master_list = []
        
        for count, word in counts:
            master_list.append((count, word))
        print len(master_list)
            
     
    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                  combiner=self.combiner,
                  reducer=self.reducer),
            MRStep(mapper=self.mapper_new,
                  reducer=self.reducer_new) ]    

if __name__ == '__main__':
    MRWordFreqCount.run()

Overwriting basket.py


In [199]:
!python basket.py browse_data.txt

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/basket.z013nx1.20160707.160901.656141
Running step 1 of 2...
Running step 2 of 2...
12592
Streaming final output from /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/basket.z013nx1.20160707.160901.656141/output...
Removing temp directory /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/basket.z013nx1.20160707.160901.656141...


12,592 unique items

Using a single reducer: Report your findings: such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,   (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 

In [196]:
%%writefile basket2.py
from mrjob.job import MRJob
from mrjob.step import MRStep
 
class MRWordFreqCount(MRJob):
    def mapper(self, word, line):
        line = line.strip().split(" ")
        for word in line:
            yield word.lower(), 1

    def combiner(self, word, counts):
        yield word, sum(counts)

    def reducer(self, word, counts):
        yield word, sum(counts)
    
    def mapper_new(self, word, counts):
        yield None, (counts, word)
 
    def reducer_new(self, word, counts):
        master_list = []
        topfifty = []
        for count, word in counts:
            master_list.append((count, word))
            master_list.sort()
            topfifty = master_list[-50:]
            topfifty.sort(reverse=True)
        print len(master_list)
        
        for count, word in topfifty:
            yield None, (count, word)

    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                  combiner=self.combiner,
                  reducer=self.reducer),
            MRStep(mapper=self.mapper_new,
                  reducer=self.reducer_new) ]    

if __name__ == '__main__':
    MRWordFreqCount.run()

Overwriting basket2.py


In [197]:
!python basket2.py browse_data.txt

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/basket2.z013nx1.20160707.155342.653907
Running step 1 of 2...
Running step 2 of 2...
12592
Streaming final output from /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/basket2.z013nx1.20160707.155342.653907/output...
null	[6667, "dai62779"]
null	[3881, "fro40251"]
null	[3875, "ele17451"]
null	[3602, "gro73461"]
null	[3044, "sna80324"]
null	[2851, "ele32164"]
null	[2736, "dai75645"]
null	[2455, "sna45677"]
null	[2330, "fro31317"]
null	[2293, "dai85309"]
null	[2292, "ele26917"]
null	[2233, "fro80039"]
null	[2115, "gro21487"]
null	[2083, "sna99873"]
null	[2004, "gro59710"]
null	[1920, "gro71621"]
null	[1918, "fro85978"]
null	[1840, "gro30386"]
null	[1816, "ele74009"]
null	[1784, "gro56726"]
null	[1773, "dai63921"]
null	[1756, "gro46854"]
null	[1713, "ele66600"]
null	[1712, "dai83733"]
null	[1702, "fro32293"]
null	[1697, "ele66810"]
null	[1646, "sna55762"]
null	[162

In [208]:
%%writefile basket3.py
from mrjob.job import MRJob
from mrjob.step import MRStep
 
class MRWordFreqCount(MRJob):

    def mapper(self, word, line):
        line = line.strip().split(" ")
        yield None, len(line)

    
    def reducer(self, word, counts):
        basketlist = []
        for length in counts:
            basketlist.append(length)
            basketlist.sort()
            basketlist = basketlist[-1:]
        print basketlist       

"""
    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                  combiner=self.combiner,
                  reducer=self.reducer),
            MRStep(mapper=self.mapper_new,
                  reducer=self.reducer_new) ]    
"""
if __name__ == '__main__':
    MRWordFreqCount.run()

Overwriting basket3.py


In [209]:
!python basket3.py browse_data.txt

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/basket3.z013nx1.20160707.161944.776493
Running step 1 of 1...
[37]
Streaming final output from /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/basket3.z013nx1.20160707.161944.776493/output...
Removing temp directory /var/folders/08/y_lkzc2n2973stvw32_hzr4h13s_fb/T/basket3.z013nx1.20160707.161944.776493...


Largest basket = 37, Unique Items = 12,592, Top 50 list see "In [197]:" above 

HW2.4. (Computationally prohibitive but then again Hadoop can handle this) Pairs

From a data mining perspective (and the aPriori algorihtm in particular), Support and Confidence are defined as follows:

       SUPPORT
       In data mining, the support value of X (where X is a collection of cooccurring items sometimes referred to as 
       an item-set. E.g., a basket or subset of a basket) with respect to T  (a transaction database where each 
       row is a transaction such as a basket of items that have been purchased)  is defined as the proportion 
       of transactions in the  database which contains  the item-set X. (a relative frequency of sorts) 

       CONFIDENCE 
       The confidence value of a rule, X ==>  Y (where X is a collection of cooccurring items and Y is generally 
       a single item. E.g., If Diapers and Beer then Cigars were also purchased), with respect to a set of transactions T, is the 
       proportion of the transactions that contains X which also contains Y. (Think of it as  tgePr(Y|X) )

       The pairs/stripes algorithm returns cooccurrence information that can be used directly to  calculate the confidence and support. 
       Note that confidence for pair X ==>  Y will  differ from the relative frequency that results from stripes when X occurs by itself in transactions.


Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a map-reduce program 
to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.

List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (number of records where they coccur, the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support  for frequent (100>count) itemsets of size 2. 

Use the Pairs pattern  to  extract these frequent itemsets of size 2. Free free to use combiners if they bring value. Instrument your code with counters for count the number of times your mapper, combiner and reducers are called.  

Please output records of the following form for the top 50 pairs (itemsets of size 2): 

      item1, item2, support count, support  (OPTIONAL Feel free to add in confidence level also)



Fix the ordering of the pairs lexicographically (left to right), 
and break ties in support (between pairs, if any exist) 
by taking the first ones in lexicographically increasing order. 

Report  the compute time for the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)
Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts.


In [5]:
##Answer

In [ ]:
%time !python BasketPairs.py Data/ProductPurchaseData.txt --top_n 50 --support_count 100

HW2.5: Stripes
Repeat 2.4 using the stripes design pattern for finding cooccuring pairs (and out.

Report  the compute times for stripes job versus the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)

Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts. Discuss the differences in these counts between the Pairs and Stripes jobs

In [6]:
##Answer